## Download the data

In [1]:
!mkdir train 
!mkdir test 
!wget http://pjreddie.com/media/files/VOCtrainval_06-Nov-2007.tar -P train/ 
!wget http://pjreddie.com/media/files/VOCtest_06-Nov-2007.tar -P test/ 
!tar -xf test/VOCtest_06-Nov-2007.tar -C test/ 
!tar -xf train/VOCtrainval_06-Nov-2007.tar -C train/ 
!rm -rf test/VOCtest_06-Nov-2007.tar

--2021-01-31 11:35:04--  http://pjreddie.com/media/files/VOCtrainval_06-Nov-2007.tar
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://pjreddie.com/media/files/VOCtrainval_06-Nov-2007.tar [following]
--2021-01-31 11:35:04--  https://pjreddie.com/media/files/VOCtrainval_06-Nov-2007.tar
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 460032000 (439M) [application/octet-stream]
Saving to: ‘train/VOCtrainval_06-Nov-2007.tar’

VOCtrainval_06-Nov- 100%[===================>] 438.72M  8.86MB/s    in 63s     

2021-01-31 11:36:07 (6.99 MB/s) - ‘train/VOCtrainval_06-Nov-2007.tar’ saved [460032000/460032000]

URL transformed to HTTPS due to an HSTS policy
--2021-01-31 11:36:07--  https://pjreddie.com/media/files/VOCtest_06-Nov-2007.tar
Resolving pjreddie.com (

In [2]:
!pip install xmltodict 
!pip install -U albumentations

     |████████████████████████████████| 81kB 9.3MB/s 
     |████████████████████████████████| 952kB 29.9MB/s 
     |████████████████████████████████| 37.6MB 85kB/s 
  Found existing installation: imgaug 0.2.9
    Uninstalling imgaug-0.2.9:
      Successfully uninstalled imgaug-0.2.9
  Found existing installation: albumentations 0.1.12
    Uninstalling albumentations-0.1.12:
      Successfully uninstalled albumentations-0.1.12


## Define path, classes

In [3]:
root_dir = '/content'
annot_f = './{}/VOCdevkit/VOC2007/Annotations'
image_f = './{}/VOCdevkit/VOC2007/JPEGImages/{}'

classes = ['person', 'bird', 'cat', 'cow', 'dog', 'horse', 
           'sheep', 'aeroplane', 'bicycle', 'boat', 'bus', 'car', 
           'motorbike', 'train', 'bottle', 'chair', 'dining table', 
           'potted plant', 'sofa', 'tv/monitor' ]

num_classes = len(classes)
feature_size = 7
num_bboxes = 2

## Import module

In [4]:
import sys

from torch.autograd import Variable
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.optim.lr_scheduler

from torch.utils.data import Dataset, DataLoader

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

## utils
import numpy as np
import random, math, time
from tqdm.notebook import tqdm

## File Loader
import os, xmltodict
import os.path as pth
from PIL import Image

# Draw Image
import matplotlib.pyplot as plt
import matplotlib.patches as patches

## Transformer
from random import sample
import albumentations as A
from albumentations.pytorch.transforms import ToTensor

# Seed
random.seed(53)

## Define visualization method

In [5]:
def draw_image(image_info, w=448, h=448, transforms=None):
    im = np.array(Image.open(image_f.format('train', image_info['image_id'])).convert('RGB').resize((w,h)), dtype=np.uint8)

    # Create figure and axes
    fig, ax  = plt.subplots(1, figsize=(7,7))

    bb = image_info['bboxs']
    la = image_info['labels']

    if transforms:
        sample = transforms(image=im, bboxes=bb, category_ids=la)
        im = sample['image'].permute(1,2,0).numpy()
        bb = sample['bboxes']
        la = sample['category_ids']

    # Display the image
    ax.imshow(im)

    for b, l in zip(bb, la): 
        # top left (x, y) , (w, h)
        rect = patches.Rectangle((b[0]*w,b[1]*h),(b[2]-b[0])*w,(b[3]-b[1])*h,linewidth=1,edgecolor='r',
                                 facecolor='none') 
        # Add the patch to the Axes 
        ax.add_patch(rect) 
        props = dict(boxstyle='round', facecolor='red', alpha=0.9) 
        plt.text(b[0]*w, b[1]*h, classes[l], fontsize=10, color='white', bbox=props) 
    plt.axis('off') 
    plt.show()

## xml파일에서 정보추출하는 함수 (xmltodict라는 모듈 사용)

In [6]:
def get_infos(annot_f=annot_f, mode='train'): 
    annot_dir = annot_f.format(mode) 
    result = [] 
    for ano in [pth.join(annot_dir, ano) for ano in os.listdir(annot_dir)]: 
        f = open(ano) 
        info = xmltodict.parse(f.read())['annotation'] 
        image_id = info['filename'] 
        image_size = np.asarray(tuple(map(int, info['size'].values()))[:2], np.int16) 
        w, h = image_size 
        box_objects = info['object'] 
        labels = [] 
        bboxs = [] 
        for obj in box_objects: 
            try: 
                labels.append(classes.index(obj['name'].lower())) 
                bboxs.append(tuple(map(int, obj['bndbox'].values()))) 
            except: pass 
        # Resizing Box, Change x1 y1 x2 y2 
        # albumentations (normalized box) 
        bboxs = np.asarray(bboxs, dtype=np.float64) 
        try: 
            bboxs[:, [0,2]] /= w 
            bboxs[:, [1,3]] /= h 
        except: pass 
        if bboxs.shape[0] or mode=='test': 
            result.append({'image_id':image_id, 'image_size':image_size, 'bboxs':bboxs, 'labels':labels}) 
    return result 
    
trval_list = get_infos() 
test_list = get_infos(mode='test') 

len(trval_list), len(test_list)

(3067, 4952)

## split train and val data

In [7]:
def get_tv_idx(tl, k=0.5):
    total_idx = range(tl) # (0,3067)
    train_idx = sample(total_idx, int(tl*k)) # 3067/2 개만큼 뽑음
    valid_idx = set(total_idx) - set(train_idx) # 안뽑힌 index들이 valid_idx
    return train_idx, list(valid_idx)

train_idx, valid_idx = get_tv_idx(len(trval_list))

trval_list = np.asarray(trval_list)  # list -> array
train_list = trval_list[train_idx]
valid_list = trval_list[valid_idx]

len(train_list), len(valid_list), len(test_list)

(1533, 1534, 4952)

## Make Dataset

In [8]:
class VOCDataset(Dataset):
    def __init__(self, data_list, mode='train', transforms=None):
        self.data_list = data_list
        self.mode = mode
        self.transforms = transforms

    def __len__(self):
        return len(self.data_list)

    def __getitem__(self, idx):
        record = self.data_list[idx]
        img_id = record['image_id']
        bboxs = record['bboxs']
        labels = record['labels']

        img = Image.open(image_f.format(self.mode, img_id)).convert('RGB')
        img = np.array(img)

        if self.transforms:
            for t in self.transforms:
                sample = self.transforms(image=img, bboxes=bboxs, category_ids=labels)
                image = sample['image']
                bboxs = np.asarray(sample['bboxes'])
                labels = np.asarray(sample['category_ids'])

        if self.mode== 'train':
            target = encode(bboxs, labels)
            return image, target
        else:
            return image

## (7,7,30) 에서 물체가 있는 곳에 1 대입(confidence, label)

In [9]:
def encode(bboxs, labels):    
    S = feature_size
    B = num_bboxes
    N = 5 * B + num_classes
    cell_size = 1.0 / float(S)

    box_cxy = (bboxs[:, 2:] + bboxs[:, :2]) / 2.0
    box_wh = (bboxs[:, 2:] - bboxs[:, :2])
    target = np.zeros((S,S,N))
    for b in range(bboxs.shape[0]): # gt박스 수만큼 반복
        cxy, wh, label = box_cxy[b], box_wh[b], labels[b]
        ij = np.ceil(cxy / cell_size) -1.0 # ceil -> 소수점있으면 무조건 올림 4.1 -> 5
        i,j = map(int, ij) # i,j는 셀 번호 0~6
        top_left = ij*cell_size # 각 셀의 좌상단 좌표
        dxy_norm = (cxy-top_left) / cell_size
        
        for k in range(B): # 한 셀당 두개의 박스
            target[i, j, 5*k:5*(k+1)] = np.r_[dxy_norm, wh, 1]  # confidence에 1
        target[j, i, 5*B+label] = 1.0 # 해당label에 1
    return target


## Albumentations

In [10]:
def get_train_transforms():
    return A.Compose([
        A.Resize(448,448, always_apply=True, p=1),
        A.RandomBrightnessContrast(p=0.2),
        A.HorizontalFlip(),
        ToTensor(),
    ],bbox_params = A.BboxParams(format='albumentations', label_fields=['category_ids']))

def get_test_transforms():
    return A.Compose([
        A.Resize(448, 448, always_apply=True, p=1),
        ToTensor(),
    ])

## Data Loader

In [11]:
train_ds = VOCDataset(train_list, transforms=get_train_transforms())
valid_ds = VOCDataset(valid_list, transforms=get_test_transforms())
test_ds = VOCDataset(test_list, mode='test', transforms=get_test_transforms())

# torch tensor를 batch size만큼 묶어줌
def collate_fn(batch):
    images, targets = zip(*batch)
    return torch.cat([img.reshape(-1, 3, 448, 448) for img in images], 0), torch.FloatTensor(targets)

def test_collate_fn(batch):
    images = batch
    return torch.cat([img.reshape(-1, 3, 448, 448) for img in images], 0)

train_loader = DataLoader(train_ds, batch_size=32, shuffle=True, collate_fn = collate_fn)
valid_loader = DataLoader(valid_ds, batch_size=32, shuffle=True, collate_fn = collate_fn)
test_loader = DataLoader(test_ds, batch_size=1, shuffle=True, collate_fn = test_collate_fn)


## Define YOLO v1 Model

In [12]:
class YOLO_v1(nn.Module):
    def __init__(self, num_classes=20, num_bboxes=2):
        super(YOLO_v1, self).__init__()

        self.feature_size = 7
        self.num_bboxes = num_bboxes
        self.num_classes = num_classes
        self.conv = nn.Sequential( 
            nn.Conv2d(in_channels=3, out_channels=64, kernel_size=7, stride=2, padding=4), 
            # nn.BatchNorm2d(64), 
            nn.LeakyReLU(0.1, inplace=True), 
            nn.MaxPool2d(kernel_size=2, stride=2), 
            
            nn.Conv2d(in_channels=64, out_channels=192, kernel_size=3, stride=1, padding=1), 
            # nn.BatchNorm2d(192), 
            nn.LeakyReLU(0.1, inplace=True), 
            nn.MaxPool2d(kernel_size=2, stride=2), 

            nn.Conv2d(in_channels=192, out_channels=128, kernel_size=1, stride=1, padding=0), 
            # nn.BatchNorm2d(128), 
            nn.LeakyReLU(0.1, inplace=True), 
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, stride=1, padding=1), 
            # nn.BatchNorm2d(256), 
            nn.LeakyReLU(0.1, inplace=True), 
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=1, stride=1, padding=0), 
            # nn.BatchNorm2d(256), 
            nn.LeakyReLU(0.1, inplace=True), 
            nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, stride=1, padding=1), 
            # nn.BatchNorm2d(512), 
            nn.LeakyReLU(0.1, inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2), 

            nn.Conv2d(in_channels=512, out_channels=256, kernel_size=1, stride=1, padding=0), 
            # nn.BatchNorm2d(256), 
            nn.LeakyReLU(0.1, inplace=True), 
            nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, stride=1, padding=1), 
            # nn.BatchNorm2d(512), 
            nn.LeakyReLU(0.1, inplace=True), 
            nn.Conv2d(in_channels=512, out_channels=256, kernel_size=1, stride=1, padding=0), 
            # nn.BatchNorm2d(256), 
            nn.LeakyReLU(0.1, inplace=True), 
            nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, stride=1, padding=1), 
            # nn.BatchNorm2d(512), 
            nn.LeakyReLU(0.1, inplace=True), 
            nn.Conv2d(in_channels=512, out_channels=256, kernel_size=1, stride=1, padding=0), 
            # nn.BatchNorm2d(256), 
            nn.LeakyReLU(0.1, inplace=True), 
            nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, stride=1, padding=1), 
            # nn.BatchNorm2d(512), 
            nn.LeakyReLU(0.1, inplace=True), 
            nn.Conv2d(in_channels=512, out_channels=256, kernel_size=1, stride=1, padding=0), 
            # nn.BatchNorm2d(256), 
            nn.LeakyReLU(0.1, inplace=True), 
            nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, stride=1, padding=1), 
            # nn.BatchNorm2d(512), 
            nn.LeakyReLU(0.1, inplace=True), 
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=1, stride=1, padding=0), 
            # nn.BatchNorm2d(512), 
            nn.LeakyReLU(0.1, inplace=True), 
            nn.Conv2d(in_channels=512, out_channels=1024, kernel_size=3, stride=1, padding=1),
            # nn.BatchNorm2d(1024), 
            nn.LeakyReLU(0.1, inplace=True), 
            nn.MaxPool2d(kernel_size=2, stride=2), 

            nn.Conv2d(in_channels=1024, out_channels=512, kernel_size=1, stride=1, padding=0), 
            # nn.BatchNorm2d(512), 
            nn.LeakyReLU(0.1, inplace=True),
            nn.Conv2d(in_channels=512, out_channels=1024, kernel_size=3, stride=1, padding=1), 
            # nn.BatchNorm2d(1024), 
            nn.LeakyReLU(0.1, inplace=True),
            nn.Conv2d(in_channels=1024, out_channels=512, kernel_size=1, stride=1, padding=0), 
            # nn.BatchNorm2d(512), 
            nn.LeakyReLU(0.1, inplace=True), 
            nn.Conv2d(in_channels=512, out_channels=1024, kernel_size=3, stride=1, padding=1), 
            # nn.BatchNorm2d(1024), 
            nn.LeakyReLU(0.1, inplace=True), 
            nn.Conv2d(in_channels=1024, out_channels=1024, kernel_size=3, stride=1, padding=1), 
            # nn.BatchNorm2d(1024), 
            nn.LeakyReLU(0.1, inplace=True), 
            nn.Conv2d(in_channels=1024, out_channels=1024, kernel_size=3, stride=2, padding=1), 
            # nn.BatchNorm2d(1024), 
            nn.LeakyReLU(0.1, inplace=True), 

            nn.Conv2d(in_channels=1024, out_channels=1024, kernel_size=3, stride=1, padding=1), 
            # nn.BatchNorm2d(1024), 
            nn.LeakyReLU(0.1, inplace=True), 
            nn.Conv2d(in_channels=1024, out_channels=1024, kernel_size=3, stride=1, padding=1), 
            # nn.BatchNorm2d(1024), 
            nn.LeakyReLU(0.1, inplace=True), 
        )

        self.fc = nn.Sequential(
            Flatten(),
            nn.Linear(in_features=7*7*1024, out_features=4096),
            nn.LeakyReLU(0.1, inplace=True),
            nn.Dropout(p=0.5),
            nn.Linear(in_features=4096, out_features=(feature_size*feature_size*(5*num_bboxes+num_classes))),
            nn.Softmax()
        )

        self.init_weight(self.conv)
        self.init_weight(self.fc)

    def forward(self, x):
        s,b,c = self.feature_size, self.num_bboxes, self.num_classes

        x = self.conv(x)
        x = self.fc(x)

        x = x.view(-1, s, s, (5 * b + c))
        return x

    def init_weight(self, modules):
        for m in modules:
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_in', nonlinearity='leaky_relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)

class Squeeze(nn.Module):
    def __init__(self):
        super(Squeeze, self).__init__()

    def forward(self, x):
        return x.squeeze()


class Flatten(nn.Module):
    def __init__(self):
        super(Flatten, self).__init__()

    def forward(self, x):
        return x.view(x.size(0), -1)

# Inter section over union

In [13]:
def compute_iou(bbox1, bbox2):
    '''
    Compute the IOU (Intersection over Union) of two set of bboxes, each bbox format: [x1,y1,x2,y2]
    :param bbox1: (Tensor) bounding boxes, sized [N,4]
    :param bbox2: (Tensor) bounding boxes, sized [N,4]
    :return: (Tensor) IoU, sized [N, M].
    '''

    N = bbox1.size(0)
    M = bbox2.size(0)

    # Compute left-top coordinate of the intersections
    lt = torch.max(
        bbox1[:, :2].unsqueeze(1).expand(N, M, 2),  # [N,2] -> [N,1,2] -> [N,M,2]
        bbox2[:, :2].unsqueeze(0).expand(N, M, 2)  # [M,2] -> [1,M,2] -> [N,M,2]
    )
    # Compute right-bottom coordinate of the intersections
    rb = torch.min(
        bbox1[:, 2:].unsqueeze(1).expand(N, M, 2),  # [N,2] -> [N,1,2] -> [N,M,2]
        bbox2[:, 2:].unsqueeze(0).expand(N, M, 2)  # [M,2] -> [1,M,2] -> [N,M,2]
    )
    # Compute area of the intersections from the coordinates
    wh = rb - lt  # width and height of the intersection, [N,M,2]
    wh[wh < 0] = 0  # clip at 0
    inter = wh[:, :, 0] * wh[:, :, 1]  # [N,M]

    # Compute area of the bboxes
    area1 = (bbox1[:, 2] - bbox1[:, 0]) * (bbox1[:, 3] - bbox1[:, 1])  # [N,]
    area2 = (bbox2[:, 2] - bbox2[:, 0]) * (bbox2[:, 3] - bbox2[:, 1])  # [N,]
    area1 = area1.unsqueeze(1).expand_as(inter)  # [N,] -> [N, 1] -> [N,M]
    area2 = area2.unsqueeze(0).expand_as(inter)  # [N,] -> [N, 1] -> [N,M]

    # Compute IoU from the areas
    union = area1 + area2 - inter
    iou = inter / union

    return iou

## Loss function

In [14]:
def loss_fn(pred_tensor, target_tensor):
    """ Compute loss for YOLO training.
    Args:
        pred_tensor: (Tensor) predictions, sized [n_batch, S, S, Bx5+C], 5=len([x, y, w, h, conf]).
        target_tensor: (Tensor) targets, sized [n_batch, S, S, Bx5+C].
    Returns:
        (Tensor): loss, sized [1, ].
    """
    S, B, C = feature_size, num_bboxes, num_classes
    N = 5 * B + C  # 5=len([x, y, w, h, conf]
    lambda_coord = 5
    lambda_noobj = 0.5

    batch_size = pred_tensor.size(0)
    coord_mask = target_tensor[:, :, :, 4] > 0  # mask for the cells which contain objects. [n_batch, S, S]
    noobj_mask = target_tensor[:, :, :, 4] == 0 # mask for the cells which not contain objects [n_batch, S, S]
    coord_mask = coord_mask.unsqueeze(-1).expand_as(target_tensor)  # [n_batch, S, S] -> [n_batch, S, S, N]
    noobj_mask = noobj_mask.unsqueeze(-1).expand_as(target_tensor)  # [n_batch, S, S] -> [n_batch, S, S, N]

    coord_pred = pred_tensor[coord_mask].view(-1, N)  # pred tensor on the cells which contain objects. [n_coord, N]
                                                        # n_coord: numver of the cells which contain objects.
    bbox_pred = coord_pred[:, :5*B].contiguous().view(-1, 5)  # [n_coord x B, 5=len([x, y, w, h, conf])]
    class_pred = coord_pred[:, 5*B:]                          # [n_coord, C]

    coord_target = target_tensor[coord_mask].view(-1, N)  # target tensor on the cells which contain objects. [n_coord, N]

    bbox_target = coord_target[:, :5*B].contiguous().view(-1, 5) # [batch*7*7*2, 5]
    class_target = coord_target[:, 5*B:]

    # Compute loss for the cells with no object bbox

    noobj_pred = pred_tensor[noobj_mask].view(-1, N)  # [n_noobj, N]
    noobj_target = target_tensor[noobj_mask].view(-1, N)

    noobj_conf_mask = torch.cuda.ByteTensor(noobj_pred.size()).fill_(0)  # [n_noobj, N]
    for b in range(B):
        noobj_conf_mask[:, 4+B*5] = 1 # noobj_conf_mask[:, 4] = 1; noobj_conf_mask[:, 9] = 1
    noobj_pred_conf = noobj_pred[noobj_conf_mask]  # [n_noobj, 2=len([conf1, conf2])]
    noobj_target_conf = noobj_target[noobj_conf_mask]
    # No object confidence loss (SSE)
    loss_noobj = F.mse_loss(noobj_pred_conf, noobj_target_conf, reduction='sum')

    # Compute loss for the cells with objects
    coord_response_mask = torch.cuda.ByteTensor(bbox_target.size()).fill_(0)  # [n_coord x B, 5]
    coord_not_response_mask = torch.cuda.ByteTensor(bbox_target.size()).fill_(1)  # [n_coord x B, 5]
    bbox_target_iou = torch.zeros(bbox_target.size()).cuda()  # [n_coord x B, 5], only the last 1=(conf,) is used

    # Choose the predicted bbox having the highest IoU for each target bbox
    for i in range(0, bbox_target.size(0), B):
        pred = bbox_pred[i:i + B]  # predicted bboxes at i-th cell, [B, 5=len([x, y, w, h, conf])]
        pred_xyxy = Variable(torch.FloatTensor(pred.size()))  # [B, 5=len([x1, y1, x2, y2, conf])]
        # Because (center_x,center_y)=pred[:, 2] and (w,h)=pred[:,2:4] are normalized for cell-size and image-size respectively,
        # rescale (center_x,center_y) for the image-size to compute IoU correctly.
        pred_xyxy[:, :2] = pred[:, :2] / float(S) - 0.5 * pred[:, 2:4]
        pred_xyxy[:, 2:4] = pred[:, :2] / float(S) + 0.5 * pred[:, 2:4]

        target = bbox_target[
            i]  # target bbox at i-th cell. Because target boxes contained by each cell are identical in current implementation, enough to extract the first one.
        target = bbox_target[i].view(-1, 5)  # target bbox at i-th cell, [1, 5=len([x, y, w, h, conf])]
        target_xyxy = Variable(torch.FloatTensor(target.size()))  # [1, 5=len([x1, y1, x2, y2, conf])]
        # Because (center_x,center_y)=target[:, 2] and (w,h)=target[:,2:4] are normalized for cell-size and image-size respectively,
        # rescale (center_x,center_y) for the image-size to compute IoU correctly.
        target_xyxy[:, :2] = target[:, :2] / float(S) - 0.5 * target[:, 2:4]
        target_xyxy[:, 2:4] = target[:, :2] / float(S) + 0.5 * target[:, 2:4]

        iou = compute_iou(pred_xyxy[:, :4], target_xyxy[:, :4])  # [B, 1]
        max_iou, max_index = iou.max(0)
        max_index = max_index.data.cuda()

        coord_response_mask[i + max_index] = 1
        coord_not_response_mask[i + max_index] = 0

        # "we want the confidence score to equal the intersection over union (IOU) between the predicted box and the ground truth"
        # from the original paper of YOLO.
        bbox_target_iou[i + max_index, torch.LongTensor([4]).cuda()] = (max_iou).data.cuda()
    bbox_target_iou = Variable(bbox_target_iou).cuda()

    # BBox location/size and objectness loss for the response bboxes.
    bbox_pred_response = bbox_pred[coord_response_mask].view(-1, 5)  # [n_response, 5]
    bbox_target_response = bbox_target[coord_response_mask].view(-1,
                                                                    5)  # [n_response, 5], only the first 4=(x, y, w, h) are used
    target_iou = bbox_target_iou[coord_response_mask].view(-1,
                                                            5)  # [n_response, 5], only the last 1=(conf,) is used
    loss_xy = F.mse_loss(bbox_pred_response[:, :2], bbox_target_response[:, :2], reduction='sum')
    loss_wh = F.mse_loss(torch.sqrt(bbox_pred_response[:, 2:4]), torch.sqrt(bbox_target_response[:, 2:4]),
                            reduction='sum')
    loss_obj = F.mse_loss(bbox_pred_response[:, 4], target_iou[:, 4], reduction='sum')

    # Class probability loss for the cells which contain objects.
    loss_class = F.mse_loss(class_pred, class_target, reduction='sum')

    # Total loss
    loss = lambda_coord * (loss_xy + loss_wh) + loss_obj + lambda_noobj * loss_noobj + loss_class
    loss = loss / float(batch_size)

    return loss

## optimizer and Learning rate scheduler

In [15]:
yolo = YOLO_v1().cuda()

init_lr = 0.001
base_lr = 0.01
optimizer = optim.SGD(yolo.parameters(), lr=init_lr, momentum=0.9, weight_decay=5e-4)

def update_lr(optimizer, epoch, burnin_base, burnin_exp=4.0):
    if epoch in range(50):
        lr = init_lr + (base_lr - init_lr) * math.pow(epoch/(50-1), burnin_exp)
    elif epoch == 50:
        lr = base_lr
    elif epoch == 750:
        lr = 0.001
    elif epoch == 1050:
        lr = 0.0001
    else:
        return

    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

## Training

In [ ]:
start_time = time.time()
bl = len(train_loader) # 48
history = {'total_loss':[]}
for epoch in range(150):
    tk0 = tqdm(train_loader, total=bl, leave=False)
    t_loss = 0.0
    breaking=False

    for step, (image, target) in enumerate(tk0):
        image, target = image.to(device), target.to(device)
        update_lr(optimizer, epoch, float(step) / float(bl - 1))
        output = yolo(image)
        loss = loss_fn(output, target).cuda()

        if math.isnan(loss):
            print(loss)
            breaking = True
            break
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        t_loss += loss.item()

        history['total_loss'].append(loss.item())

    if breaking:
        break

    # print statistics
    tqdm.write(f'[EPOCH : {epoch+1} total_loss: {t_loss/bl} Total_elapesd_time: {(time.time()-start_time)/60}분')

    state = {'epoch': epoch,
             'model': yolo,
             'optimizer': optimizer}
    filename = 'checkpoint_yolov1.pth.tar'
    torch.save(state, filename)

print(time.time() - start_time)
print('Finished Training')


## 예측결과 Tensor를 다시 boxes, labels, confidences, class_score로 decoding하는 함수

In [24]:
def decode(pred_tensor):
    '''
    Decode tensor into box coordinates, class labels, and probs_detected.
    Args:
        pred_tensor: (tensor) tensor to decode sized [S, S, 5 x B + C], 5=(x, y, w, h, conf)
    Returns:
        boxes: (tensor) [[x1, y1, x2, y2]_obj, ...]. Normalized from 0.0 to 1.0 w.r.t. image width/height, sized [n_boxes, 4].
        labels: (tensor) class labels for each detected box, sized [n_boxes]
        confidences: (tensor) objectness confidences for each detected box, sized [n_boxes].
        class_score: (tensor) scores for most likely class for each detected box, sized [n_boxes,].
    '''    
    S,B,C = feature_size, num_bboxes, num_classes
    conf_thresh = 0.1
    prob_thresh = 0.1
    nms_thresh = 0.5

    boxes, labels, confidences, class_scores = [], [], [], []

    cell_size = 1.0 / float(S)

    conf = pred_tensor[:,:,4].unsqueeze(2)  # [7, 7, 1]
    for b in range(1,B):
        conf = torch.cat((conf, pred_tensor[:,:,5*b+4].unsqueeze(2)), 2)
    conf_mask = conf > conf_thresh # [S, S, B]

    # TBM, further optimization may be possible by replacing thre following for-loops with tensor opterations.
    for i in range(S): # for x-dimension.
        for j in range(S): # for y_dimension.
            class_score, class_label = torch.max(pred_tensor[j, i, 5*B:], 0) # 해당 셀에서 가장 높은 class score
            
            for b in range(B):
                conf = pred_tensor[j, i, 5*b+4]
                prob = conf * class_score
                if float(prob) < prob_thresh:
                    continue
                
                # Compute box corner (x1, y1, x2, y2) from tensor.
                box = pred_tensor[j, i, 5*b : 5*b+4]
                # cell left-top corner. Normalized from 0.0 to 1.0 w.r.t. image width/height
                x0y0_normalized = torch.FloatTensor([i,j]) * cell_size 
                # box center. Normalized from 0.0 to 1.0 w.r.t. image width/height.
                xy_normalized = box[:2] * cell_size + x0y0_normalized
                # Box width and height. Normalized from 0.0 to 1.0 w.r.t. image width/height.
                wh_normalized = box[2:]

                box_xyxy = torch.FloatTensor(4) # [4,]
                box_xyxy[:2] = xy_normalized - 0.5 * wh_normalized # left-top corner(x1, y1)
                box_xyxy[2:] = xy_normalized + 0.5 * wh_normalized # right-bottom corner(x2, y2)

                # Append result to the lists
                boxes.append(box_xyxy)
                labels.append(class_label)
                confidences.append(conf)
                class_score.append(class_score)

    if len(boxes) > 0:
        boxes = torch.stack(boxes, 0) # [n_boxes, 4]
        labels = torch.stack(labels, 0) # [n_boxes]
        confidences = torch.stack(confidences, 0) # [n_boxes]
        class_score = torch.stack(class_score, 0) # [n_boxes]
    else:
        # If no box found, return empty tensors.
        boxes = torch.FloatTensor(0, 4) 
        labels = torch.LongTensor(0) 
        confidences = torch.FloatTensor(0) 
        class_scores = torch.FloatTensor(0)

    return boxes, labels, confidences, class_score

## Test Visualization

In [31]:
def test_visualize(images, outputs):
    fig, ax = plt.subplots(1, figsize=(7,7))
    img = Image.open(image_f.format('test',test_list[0]['image_id']))
    w,h = test_list[0]['image_size']
    im = np.asarray(img)

    ax.imshow(im)

    for output in outputs:
        b, l, c, sc = decode(output)
        if b.shape[0]: 
            # patches.Rectangle(xy,width,height)
            rect = patches.Rectangle((b[0]*w,b[1]*h),(b[2]-b[0])*w,(b[3]-b[1])*h,linewidth=1,edgecolor='r',facecolor='none') 
            ax.add_path(rect) 
            probs = dict(boxstyle='round', facecolor='red', alpha=.9) 
            plt.text(b[0]*w, b[1]*h, '%s : %.2f'%(classes[l], sc), fontsize=10, color='white', bbox=props)
